In [43]:
import sys,os
sys.path.append("..")

MONGO_URL="localhost:27017"
os.environ["MONGO_URL"] = MONGO_URL

MONGO_INITDB_ROOT_USERNAME="root" # 
os.environ["MONGO_INITDB_ROOT_USERNAME"] = MONGO_INITDB_ROOT_USERNAME

MONGO_INITDB_ROOT_PASSWORD="dr5[gnhn" # 
os.environ["MONGO_INITDB_ROOT_PASSWORD"] = MONGO_INITDB_ROOT_PASSWORD

DATABASE_NAME = "admin"
os.environ["MONGO_INITDB_DATABASE"] = DATABASE_NAME

MONGO_BOT_USERNAME="bot"
os.environ["MONGO_BOT_USERNAME"] = MONGO_BOT_USERNAME

MONGO_BOT_PASSWORD="botpsswd"
os.environ["MONGO_BOT_PASSWORD"] = MONGO_BOT_PASSWORD

# timerbot_credentials = dict(
#     DATABASE_NAME = "timerbot_db",
#     DATABASE_HOST = MONGO_URL,
#     DATABASE_ADMIN_USERNAME = MONGO_BOT_USERNAME,
#     DATABASE_ADMIN_PASSWORD = MONGO_BOT_PASSWORD
# )

# root_credentials = dict(
#     DATABASE_NAME = "timerbot_db",
#     DATABASE_HOST = MONGO_URL,
#     DATABASE_ADMIN_USERNAME = MONGO_INITDB_ROOT_USERNAME,
#     DATABASE_ADMIN_PASSWORD = MONGO_INITDB_ROOT_PASSWORD
# )

# database credentials extractor import
from modules.db.dbconfig import root_credentials, timerbot_credentials
from modules.db.mongodb import MongoDriver

db_instances = dict(root=MongoDriver(root_credentials), timerbot_db=MongoDriver(timerbot_credentials))
print(db_instances)
print(root_credentials)
print(timerbot_credentials)

# client = MongoDriver(root_credentials).client
# db_instance = MongoDriver(root_credentials)
# collection_name = "timer_data_collection"
# user_id = 122267418
# query = {"user_id": user_id}

# print(client)
# client.list_database_names()

{'root': <modules.db.mongodb.MongoDriver object at 0x000002BE762A4E20>, 'timerbot_db': <modules.db.mongodb.MongoDriver object at 0x000002BE762A4EB0>}
{'DATABASE_NAME': 'timerbot_db', 'DATABASE_HOST': 'localhost:27017', 'DATABASE_ADMIN_USERNAME': 'root', 'DATABASE_ADMIN_PASSWORD': 'dr5[gnhn'}
{'DATABASE_NAME': 'timerbot_db', 'DATABASE_HOST': 'localhost:27017', 'DATABASE_ADMIN_USERNAME': 'bot', 'DATABASE_ADMIN_PASSWORD': 'botpsswd'}


In [8]:
from modules.db.dbmodel import get_records, get_timerdata_object, purge, add_records
data = {'_id': 122267418, 'user_id': 122267418, 'username': '@radical_subject', 'firstname': 'Oleg', 'lastname': 'Fedorov'}
# collection_name = "users_collection"

In [48]:
# get_timerdata_object(client, db_instance, collection_name, query, user_id).export()

NameError: name 'client' is not defined

In [15]:
# get_records(client, db_instance, collection_name, query, user_id)
# purge(client, db_instance)
# add_records(client, db_instance, collection_name, data)

In [16]:
# collection = client["bot_db"]["users_collection"]

# data = {'_id': 122267418, 'user_id': 122267418, 'username': '@radical_subject', 'firstname': 'Oleg', 'lastname': 'Fedorov'}

In [27]:
# print(collection.insert_one(data))

In [49]:
# conda list

In [50]:
# db_instances = dict(root=MongoDriver(root_credentials), timerbot_db=MongoDriver(timerbot_credentials))
from modules.ourbot.handlers.initial import Inital
from modules.db.dbmodel import get_records, get_timerdata_object, purge, add_records

In [51]:
initial = Inital(db_instances)

In [52]:
initial.timerbot_db_client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [53]:
add_records(initial.timerbot_db_client, initial.db_instances["timerbot_db"], initial.collection, data)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение, Timeout: 30s, Topology Description: <TopologyDescription id: 623af54055d7015da9bf5d97, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение')>]>

In [ ]:
purge(initial.root_client, initial.db_instances["timerbot_db"])